In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


Use df from previous task 

In [2]:
df = pd.read_csv("Week3 Task2.csv")
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Define geograpical coordinate for Toronto

In [3]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

geolocator = Nominatim(timeout=3)
location = geolocator.geocode('Toronto, Canada')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/Users/ibm-jti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [4]:
df_toronto = df[ df.Borough.str.contains('Toronto') ]
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Visualise the Toronto's neighborhood using latitude and longitude values

In [5]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [6]:
CLIENT_ID = 'OIMP2UK0HDPGRBD1RXRT4WA4NLL35RVIAHPFXHL2UIIORP1V'
CLIENT_SECRET = 'FGUMF2EP3WMNEAU1132K22XHCI4VKNXRGGFFDJDBSVZZAM0F' 
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: OIMP2UK0HDPGRBD1RXRT4WA4NLL35RVIAHPFXHL2UIIORP1V
CLIENT_SECRET:FGUMF2EP3WMNEAU1132K22XHCI4VKNXRGGFFDJDBSVZZAM0F


## Search for a specific venue

I'll try to looking for coffee shop in Toronto's neighborhood

In [7]:
search_query = 'Coffee'
radius = 500
print(search_query + ' .... OK!')

Coffee .... OK!


Define the URL

In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=OIMP2UK0HDPGRBD1RXRT4WA4NLL35RVIAHPFXHL2UIIORP1V&client_secret=FGUMF2EP3WMNEAU1132K22XHCI4VKNXRGGFFDJDBSVZZAM0F&ll=43.653963,-79.387207&v=20180604&query=Coffee&radius=500&limit=30'

Send the GET requests

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d4bc389787dba0036822c0e'},
 'response': {'venues': [{'id': '59f784dd28122f14f9d5d63d',
    'name': 'HotBlack Coffee',
    'location': {'address': '245 Queen Street West',
     'crossStreet': 'at St Patrick St',
     'lat': 43.65036434800487,
     'lng': -79.38866907575726,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65036434800487,
       'lng': -79.38866907575726}],
     'distance': 417,
     'postalCode': 'M5V 1Z4',
     'cc': 'CA',
     'neighborhood': 'Entertainment District',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['245 Queen Street West (at St Patrick St)',
      'Toronto ON M5V 1Z4',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'pr

Create dataframe consist of coffee shop venue

In [10]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df_venues = json_normalize(venues)
df_venues.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,59f784dd28122f14f9d5d63d,245 Queen Street West,CA,Toronto,Canada,at St Patrick St,417,"[245 Queen Street West (at St Patrick St), Tor...","[{'label': 'display', 'lat': 43.65036434800487...",43.650364,-79.388669,Entertainment District,M5V 1Z4,ON,HotBlack Coffee,v-1565246345,463001529
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,53e8acc4498ee294fb100183,425 University Ave,CA,Toronto,Canada,Dundas,39,"[425 University Ave (Dundas), Toronto ON M5G 1...","[{'label': 'display', 'lat': 43.65427, 'lng': ...",43.654270,-79.387448,NaN,M5G 1T6,ON,Timothy's World Coffee,v-1565246345,NaN
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4c19447c4ff90f4765ac0f49,"595 Bay St,Atrium On Bay",CA,Toronto,Canada,at Atrium on Bay,403,"[595 Bay St,Atrium On Bay (at Atrium on Bay), ...","[{'label': 'display', 'lat': 43.656219, 'lng':...",43.656219,-79.383290,NaN,M5G 2C2,ON,Timothy's World Coffee,v-1565246345,NaN
3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4b44fc77f964a520cc0026e3,427 University Avenue,CA,Toronto,Canada,NaN,71,"[427 University Avenue, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65405317976302...",43.654053,-79.388090,NaN,NaN,ON,Timothy's World Coffee,v-1565246345,NaN
4,[],False,4baa31def964a52037523ae3,350 Bay St - 7th Floor,CA,Toronto,Canada,NaN,500,"[350 Bay St - 7th Floor, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.649498, 'lng':...",43.649498,-79.386479,NaN,NaN,ON,Coffee office,v-1565246345,NaN


Filter and clean the previous dataframe

In [11]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_venues.columns if col.startswith('location.')] + ['id']
df_coffee = df_venues.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_coffee['categories'] = df_coffee.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_coffee.columns = [column.split('.')[-1] for column in df_coffee.columns]

df_coffee

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,HotBlack Coffee,Coffee Shop,245 Queen Street West,CA,Toronto,Canada,at St Patrick St,417,"[245 Queen Street West (at St Patrick St), Tor...","[{'label': 'display', 'lat': 43.65036434800487...",43.650364,-79.388669,Entertainment District,M5V 1Z4,ON,59f784dd28122f14f9d5d63d
1,Timothy's World Coffee,Coffee Shop,425 University Ave,CA,Toronto,Canada,Dundas,39,"[425 University Ave (Dundas), Toronto ON M5G 1...","[{'label': 'display', 'lat': 43.65427, 'lng': ...",43.654270,-79.387448,NaN,M5G 1T6,ON,53e8acc4498ee294fb100183
2,Timothy's World Coffee,Coffee Shop,"595 Bay St,Atrium On Bay",CA,Toronto,Canada,at Atrium on Bay,403,"[595 Bay St,Atrium On Bay (at Atrium on Bay), ...","[{'label': 'display', 'lat': 43.656219, 'lng':...",43.656219,-79.383290,NaN,M5G 2C2,ON,4c19447c4ff90f4765ac0f49
3,Timothy's World Coffee,Coffee Shop,427 University Avenue,CA,Toronto,Canada,NaN,71,"[427 University Avenue, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65405317976302...",43.654053,-79.388090,NaN,NaN,ON,4b44fc77f964a520cc0026e3
4,Coffee office,None,350 Bay St - 7th Floor,CA,Toronto,Canada,NaN,500,"[350 Bay St - 7th Floor, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.649498, 'lng':...",43.649498,-79.386479,NaN,NaN,ON,4baa31def964a52037523ae3
5,Timothy's World Coffee,Coffee Shop,"483 Bay St,Bell Trinity Square",CA,Toronto,Canada,Bell Trinity Square,398,"[483 Bay St,Bell Trinity Square (Bell Trinity ...","[{'label': 'display', 'lat': 43.653436, 'lng':...",43.653436,-79.382314,NaN,M5G 2C9,ON,4b0aaa8ef964a520272623e3
6,Timothy's World Coffee,Coffee Shop,401 Bay St.,CA,Toronto,Canada,at Richmond St. W,526,"[401 Bay St. (at Richmond St. W), Toronto ON M...","[{'label': 'display', 'lat': 43.65213455850074...",43.652135,-79.381172,NaN,M5H 2Y4,ON,4baa9f6cf964a520817a3ae3
7,T.A.N. Coffee,Coffee Shop,37 Baldwin St,CA,Toronto,Canada,Henry St,559,"[37 Baldwin St (Henry St), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65602860741956...",43.656029,-79.393534,NaN,NaN,ON,4d8a7096d85f3704d05afedb
8,The Library Specialty Coffee,Coffee Shop,281 Dundas St West,CA,Toronto,Canada,St Patrick And Dundas St W,301,[281 Dundas St West (St Patrick And Dundas St ...,"[{'label': 'display', 'lat': 43.65441282740799...",43.654413,-79.390902,NaN,M5T 2W5,ON,5a6b737b35f98359eed11974
9,Jimmy's Coffee,Coffee Shop,82 Gerrard Street W,CA,Toronto,Canada,Gerrard & LaPlante,512,"[82 Gerrard Street W (Gerrard & LaPlante), Tor...","[{'label': 'display', 'lat': 43.65842123574496...",43.658421,-79.385613,NaN,M5G 1Z4,ON,537d4d6d498ec171ba22e7fe


Let's see only coffee shop name in Toronto's neighborhood

In [12]:
df_coffee.name

0                                       HotBlack Coffee
1                                Timothy's World Coffee
2                                Timothy's World Coffee
3                                Timothy's World Coffee
4                                         Coffee office
5                                Timothy's World Coffee
6                                Timothy's World Coffee
7                                         T.A.N. Coffee
8                          The Library Specialty Coffee
9                                        Jimmy's Coffee
10                            Coffee Antidote 9th Floor
11                                Seattle's Best Coffee
12                                       Jimmy's Coffee
13                                    Urbana Coffee Co.
14                           Fresh Start Coffee Company
15                                       Mos Mos Coffee
16                                 Union Coffee @ OCADU
17                                      Carbonic

In [13]:
coffee_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Toronto

# add a red circle marker to represent the location of Toronto
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_map)

# add the coffee shop around Toronto as blue circle markers
for lat, lng, label in zip(df_coffee.lat, df_coffee.lng, df_coffee.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(coffee_map)

# display map
coffee_map

## Explore a Given Venue 

Explore one of coffee shop in Toronto : Hotblack Coffee

In [14]:
venue_id = '59f784dd28122f14f9d5d63d' # ID of Hotblack Coffee
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/59f784dd28122f14f9d5d63d?client_id=OIMP2UK0HDPGRBD1RXRT4WA4NLL35RVIAHPFXHL2UIIORP1V&client_secret=FGUMF2EP3WMNEAU1132K22XHCI4VKNXRGGFFDJDBSVZZAM0F&v=20180604'

In [15]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'venuePage', 'reasons', 'description', 'storeId', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])


{'id': '59f784dd28122f14f9d5d63d',
 'name': 'HotBlack Coffee',
 'contact': {'phone': '4168406141',
  'formattedPhone': '(416) 840-6141',
  'twitter': 'hotblack_coffee',
  'instagram': 'hotblack_coffee'},
 'location': {'address': '245 Queen Street West',
  'crossStreet': 'at St Patrick St',
  'lat': 43.65036434800487,
  'lng': -79.38866907575726,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.65036434800487,
    'lng': -79.38866907575726}],
  'postalCode': 'M5V 1Z4',
  'cc': 'CA',
  'neighborhood': 'Entertainment District',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['245 Queen Street West (at St Patrick St)',
   'Toronto ON M5V 1Z4',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/hotblack-coffee/59f784dd28122f14f9d5d63d',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/

Find out rating for Hotblack Coffee in Foursquare

In [16]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.0


Find out how many tips from users for Hotblack Coffe

In [17]:
result['response']['venue']['tips']['count']

29

In [18]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d4bc3fdeceb65002544313b'},
 'response': {'tips': {'count': 29,
   'items': [{'id': '5851d3c36e607514d9a41b62',
     'createdAt': 1481757635,
     'text': 'Coffee taken to new level of perfection. They have a crazy water purification system and their machines pressure is controlled every second...  best experience! Good sweets.Friendly staff!',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5851d3c36e607514d9a41b62',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 3,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '7658576',
      'firstName': 'mundi',
      'lastName': 'salmanovici',
      'gender': 'male',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/7658576_UtvB2iSc_zuA8PGP1eaebLEibR1juv_55RN4YbidEupSrfEpRXEEAuHUIxlYZLFFDn_yuXauW.jpg'}}}]}}}

In [19]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

Show tips from user in dataframe

In [20]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Coffee taken to new level of perfection. They have a crazy water purification system and their machines pressure is controlled every second... best experience! Good sweets.Friendly staff!,3,0,5851d3c36e607514d9a41b62,mundi,salmanovici,male,7658576


# Search a Foursquare user

From previous tips for HotBlack Coffee, I'll pick 1 user ID and explore that user

In [21]:
user_id = '7658576' 

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

Find out name and home city for the user

In [22]:
print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])

First Name: mundi
Last Name: salmanovici
Home City: Israel


Count how many tips that posted from that user

In [23]:
user_data['tips']

{'count': 26}

Show all tips posted by the user for all places in Foursquare

In [24]:
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,"The burger is great, as well as the sweet potato fries. The whole deal is amazing for the quality you're getting.",1,0,5c87ced616fa040038d0b0ac
1,Dim sum with chicken soup is great,0,0,5ae27cf9a0215b0039e83911
2,Godiva banana muffin is great. Also the flat white was professionally executed and very good.,0,0,5a10837d1ffed7149586b51d
3,"Great atmosphere,good food, prime location. A big variety of Spanish dishes. Take a walk inside the building, what once was a parking.",0,0,59eb7aadc824ae5422eeec37
4,Coffee is very good! Nice sandwiches for 2$,2,0,5996f858c8b2fb0190d45b96
5,Unbelievable pizza!,0,0,596ce7629cadd91d4096750f
6,"Good atmosphere, very good coffee but pricey!",0,0,5964053c4420d809d05dae4b
7,"Very good coffee! Cool place with friendly staff,quiet athmosphere.",0,0,59085bcff8cbd452f54f040e
8,"Nice place,good athmosphere,but average coffee,overpriced!",0,1,58faa2d81cf2e16a0451fc25
9,Coffee taken to new level of perfection. They have a crazy water purification system and their machines pressure is controlled every second... best experience! Good sweets.Friendly staff!,3,0,5851d3c36e607514d9a41b62


In [25]:
tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

Cowgirl
{'address': '519 Hudson St', 'crossStreet': 'at W 10th St', 'lat': 40.73373338282062, 'lng': -74.0062998849649, 'labeledLatLngs': [{'label': 'display', 'lat': 40.73373338282062, 'lng': -74.0062998849649}], 'postalCode': '10014', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['519 Hudson St (at W 10th St)', 'New York, NY 10014', 'United States']}


Let's see user's friends in Foursquare

In [26]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

,bio,firstName,gender,homeCity,id,lastName,lists.groups,photo.default,photo.prefix,photo.suffix,tips.count
0,,Giulia,female,"Suceava, Romania",15497209,Cornea,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/15497209-H25DCPUJF4RQMAL2.jpg,0
1,,Julio,male,"São Paulo, Brazil",72314425,Bua,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/72314425-3UF34GAJXIJI1ALJ.jpg,0
2,,Seffi,male,"Los Angeles, CA",65024427,Lobel,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/J5DGTYPGZQ2CLFD4.jpg,0
3,,Colacei,female,"Bucharest, Romania",64788280,Mirela,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/QQFYO0SXSZRBOQJH.jpg,2
4,I wanna be myself,Tresa,female,New York,61679429,Pena,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/NLUZ0GQE2RGEDMBP.png,0
5,Show me the moon,Elizabeth,female,New York,61675917,Farr,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/DGMVQBWKFID1JXYQ.png,0
6,,Gilad,male,"86 Shlomo hamelech st. Apt 7, TLV",57511093,Yizhaki,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/G3FGUPK03VIXQVHU.jpg,0
7,,Yuliah,female,Ramat HaSharon,58185521,Vine,"[{'type': 'created', 'count': 3, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/58185521-4M0DZGKK0XRIT31N.jpg,0
8,,Helen,female,"New York, NY",56234515,Ashkenasi,"[{'type': 'created', 'count': 2, 'items': []}]",True,https://fastly.4sqi.net/img/user/,/blank_girl.png,0
9,,Doina,female,Tel Aviv,48506843,Moseanu,"[{'type': 'created', 'count': 2, 'items': []}]",True,https://fastly.4sqi.net/img/user/,/blank_girl.png,0


In [27]:
user_data

{'id': '7658576',
 'firstName': 'mundi',
 'lastName': 'salmanovici',
 'gender': 'male',
 'canonicalUrl': 'https://foursquare.com/user/7658576',
 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
  'suffix': '/7658576_UtvB2iSc_zuA8PGP1eaebLEibR1juv_55RN4YbidEupSrfEpRXEEAuHUIxlYZLFFDn_yuXauW.jpg'},
 'friends': {'count': 27,
  'groups': [{'type': 'others',
    'name': 'Other friends',
    'count': 27,
    'items': [{'id': '15497209',
      'firstName': 'Giulia',
      'lastName': 'Cornea',
      'gender': 'female',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/15497209-H25DCPUJF4RQMAL2.jpg'},
      'tips': {'count': 0},
      'lists': {'groups': [{'type': 'created', 'count': 2, 'items': []}]},
      'homeCity': 'Suceava, Romania',
      'bio': '',
      'contact': {}},
     {'id': '72314425',
      'firstName': 'Julio',
      'lastName': 'Bua',
      'gender': 'male',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix':

Retrieve user's image 

In [28]:
from IPython.display import Image
from IPython.core.display import HTML 

Image(url='https://fastly.4sqi.net/img/user/300x300/7658576_UtvB2iSc_zuA8PGP1eaebLEibR1juv_55RN4YbidEupSrfEpRXEEAuHUIxlYZLFFDn_yuXauW.jpg')

## Explore a Location Around Hotblack Coffee

In [29]:
#Hotblack Coffee location
latitude = 43.650364
longitude = -79.388669

In [30]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=OIMP2UK0HDPGRBD1RXRT4WA4NLL35RVIAHPFXHL2UIIORP1V&client_secret=FGUMF2EP3WMNEAU1132K22XHCI4VKNXRGGFFDJDBSVZZAM0F&ll=43.650364,-79.388669&v=20180604&radius=500&limit=30'

In [31]:
results = requests.get(url).json()
'There are {} places around Hotblack Coffee.'.format(len(results['response']['groups'][0]['items']))

'There are 30 places around Hotblack Coffee.'

In [32]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4e31b74252b131dcebb08743',
  'name': 'Shangri-La Toronto',
  'location': {'address': '188 University Ave.',
   'crossStreet': 'at Adelaide St. W',
   'lat': 43.64912919417502,
   'lng': -79.3865566853963,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.64912919417502,
     'lng': -79.3865566853963}],
   'distance': 218,
   'postalCode': 'M5H 0A3',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['188 University Ave. (at Adelaide St. W)',
    'Toronto ON M5H 0A3',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d1fa931735',
    'name': 'Hotel',
    'pluralName': 'Hotels',
    'shortName': 'Hotel',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []

In [33]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Shangri-La Toronto,Hotel,188 University Ave.,CA,Toronto,Canada,at Adelaide St. W,218,"[188 University Ave. (at Adelaide St. W), Toronto ON M5H 0A3, Canada]","[{'label': 'display', 'lat': 43.64912919417502, 'lng': -79.3865566853963}]",43.649129,-79.386557,NaN,M5H 0A3,ON,4e31b74252b131dcebb08743
1,Friendly Stranger - Cannabis Culture Shop,Smoke Shop,241 Queen St. W,CA,Toronto,Canada,at St. Patrick St.,11,"[241 Queen St. W (at St. Patrick St.), Toronto ON M5V 1Z4, Canada]","[{'label': 'display', 'lat': 43.650386666114635, 'lng': -79.38852348995324}]",43.650387,-79.388523,NaN,M5V 1Z4,ON,4b7ed424f964a5208a0230e3
2,JaBistro,Sushi Restaurant,222 Richmond St W,CA,Toronto,Canada,NaN,88,"[222 Richmond St W, Toronto ON M5V 1W4, Canada]","[{'label': 'display', 'lat': 43.64968685893743, 'lng': -79.38809001547467}]",43.649687,-79.388090,NaN,M5V 1W4,ON,509bb871e4b09c7ac93f6642
3,Four Seasons Centre for the Performing Arts,Concert Hall,145 Queen St. W,CA,Toronto,Canada,at University Ave.,231,"[145 Queen St. W (at University Ave.), Toronto ON M5H 4G1, Canada]","[{'label': 'display', 'lat': 43.650592, 'lng': -79.385806}]",43.650592,-79.385806,NaN,M5H 4G1,ON,4ad4c062f964a520e5f720e3
4,Copacabana Grilled Brazilian,Brazilian Restaurant,230 Adelaide St W #2,CA,Toronto,Canada,at Duncan St.,229,"[230 Adelaide St W #2 (at Duncan St.), Toronto ON M5H 3H1, Canada]","[{'label': 'display', 'lat': 43.64833259480477, 'lng': -79.3881514766071}]",43.648333,-79.388151,NaN,M5H 3H1,ON,52ec621e498ec68fa15ee922
5,Soho House Toronto,Speakeasy,192 Adelaide St W,CA,Toronto,Canada,at Simcoe St.,249,"[192 Adelaide St W (at Simcoe St.), Toronto ON M5H 0A4, Canada]","[{'label': 'display', 'lat': 43.64873430659662, 'lng': -79.38654117564838}]",43.648734,-79.386541,NaN,M5H 0A4,ON,50322b6ae4b09116a296568c
6,Pai,Thai Restaurant,18 Duncan St,CA,Toronto,Canada,Adelaide and Duncan,271,"[18 Duncan St (Adelaide and Duncan), Toronto ON M5H 3G6, Canada]","[{'label': 'display', 'lat': 43.64792310735613, 'lng': -79.38857932631602}]",43.647923,-79.388579,Entertainment District,M5H 3G6,ON,529612de11d2ab526191ccc9
7,Umbra Concept Store,Furniture / Home Store,165 John St.,CA,Toronto,Canada,at Queen St. W,198,"[165 John St. (at Queen St. W), Toronto ON M5T 1X3, Canada]","[{'label': 'display', 'lat': 43.650417, 'lng': -79.391136}]",43.650417,-79.391136,NaN,M5T 1X3,ON,4ae734bef964a5205ea921e3
8,Burrito Boyz,Burrito Place,224 Adelaide St W,CA,Toronto,Canada,btwn Duncan & Simcoe,220,"[224 Adelaide St W (btwn Duncan & Simcoe), Toronto ON M5H 1W7, Canada]","[{'label': 'display', 'lat': 43.6484455955513, 'lng': -79.3879951787513}]",43.648446,-79.387995,NaN,M5H 1W7,ON,4ad4c061f964a520a9f720e3
9,The Burger's Priest,Burger Joint,212 Adelaide Street West,CA,Toronto,Canada,NaN,234,"[212 Adelaide Street West, Toronto ON M5H 1W7, Canada]","[{'label': 'display', 'lat': 43.648476440165176, 'lng': -79.38737257506915}]",43.648476,-79.387373,NaN,M5H 1W7,ON,5578720f498ee3b165b1f32b


Visualize venue around HotBlack Coffee

In [34]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around HotBlack Coffee


# add HotBlack Coffee as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

## Get Similar Venue

Find similar venue like Hotblack Coffee for next trip's recommendation

In [37]:
# define URL
url = 'https://api.foursquare.com/v2/venues/59f784dd28122f14f9d5d63d/similar?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d4bc4828ad62e0039a0364e'},
 'response': {'similarVenues': {'count': 5,
   'items': [{'id': '537d4d6d498ec171ba22e7fe',
     'name': "Jimmy's Coffee",
     'location': {'address': '82 Gerrard Street W',
      'crossStreet': 'Gerrard & LaPlante',
      'lat': 43.65842123574496,
      'lng': -79.38561319551111,
      'labeledLatLngs': [{'label': 'display',
        'lat': 43.65842123574496,
        'lng': -79.38561319551111}],
      'distance': 930,
      'postalCode': 'M5G 1Z4',
      'cc': 'CA',
      'city': 'Toronto',
      'state': 'ON',
      'country': 'Canada',
      'formattedAddress': ['82 Gerrard Street W (Gerrard & LaPlante)',
       'Toronto ON M5G 1Z4',
       'Canada']},
     'categories': [{'id': '4bf58dd8d48988d1e0931735',
       'name': 'Coffee Shop',
       'pluralName': 'Coffee Shops',
       'shortName': 'Coffee Shop',
       'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
        'suffix': '.png'},
  

In [38]:
similar = results['response']['similarVenues']['items']

similar = results['response']['similarVenues']['items'][0]
similar.keys()

dict_keys(['id', 'name', 'location', 'categories'])

In [39]:
pd.set_option('display.max_colwidth', -1)

similar_df = json_normalize(similar) # json normalize tips

# columns to keep
filtered_columns = ['id', 'name', 'categories']
similar_filtered = similar_df.loc[:, filtered_columns]

# display tips
similar_filtered

,id,name,categories
0,537d4d6d498ec171ba22e7fe,Jimmy's Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'Coffee Shop', 'pluralName': 'Coffee Shops', 'shortName': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}, 'primary': True}]"
